In [0]:
# 导入工具包 numpy pandas用于数据处理  
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn import preprocessing
%matplotlib inline
#装载云盘
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#读取数据
df = pd.read_csv('/content/drive/My Drive/colab/5G_contest/train.csv')
#df = shuffle(df)
train=df.iloc[0:500000,]
test=df.iloc[500000:,]
train.head()

In [0]:
train.shape
train.info()
train['is_5g'].value_counts()

(500000, 60)

In [0]:
from sklearn.preprocessing import StandardScaler
#不用的特征
#进行数据归一化 mu=0 sigma=1
is_5g=df.is_5g
nouse=['area_id','user_id','prov_id','is_5g']
df=df.drop(nouse,axis=1)
df01= StandardScaler().fit_transform(df)
df= pd.DataFrame(df01,columns=df.columns)
df=pd.concat([df,is_5g],axis=1)
df.head()

,chnl_type,service_type,product_type,innet_months,total_times,total_flux,total_fee,pay_fee,sex,age,manu_name,term_type,max_rat_flag,is_5g_base_cover,is_work_5g_cover,is_home_5g_cover,is_work_5g_cover_l01,is_home_5g_cover_l01,is_work_5g_cover_l02,is_home_5g_cover_l02,activity_type,is_act_expire,comp_type,call_days,re_call10,short_call10,long_call10,bank_cnt,active_days01,active_days02,active_days03,active_days04,active_days05,active_days06,active_days07,active_days08,active_days09,active_days10,active_days11,active_days12,active_days13,active_days14,active_days15,active_days16,active_days17,active_days18,active_days19,active_days20,active_days21,active_days22,active_days23,game_app_flux,live_app_flux,video_app_flux,city_5g_ratio,city_level,is_5g
0,-0.077967,0.194582,1.025636,-0.501740,0.086904,0.871710,0.013110,-0.070508,0.699847,-0.877628,-0.541814,0.120454,0.157091,-0.591873,-0.441417,-0.430749,-0.369549,-0.365935,-0.459401,-0.452531,0.053326,-0.491182,-0.492897,0.607536,-0.425269,0.204932,-0.415563,-0.269017,-0.467869,-0.198135,-0.090953,-0.005188,-0.25889,-0.091427,-0.128203,-0.298628,-0.416863,-0.089809,0.503125,-0.261982,-0.151513,-0.014015,-0.381082,-0.517516,-0.049271,-0.13781,0.952678,-0.874288,-0.58450,-0.578889,-0.43054,-0.009756,-0.035183,0.129473,0.025871,0.920834,1.0
1,-0.077967,0.194582,-0.661127,-1.046907,-0.254138,-0.879545,-0.041040,-0.035024,0.699847,-0.784659,0.851247,0.120454,0.157091,-0.591873,-0.441417,-0.430749,-0.369549,-0.365935,-0.459401,-0.452531,0.053326,-0.491182,-0.492897,0.142525,-0.308411,-0.034857,0.024266,-0.674088,-1.093348,-0.198135,-0.090953,-0.005188,-0.25889,-0.091427,-0.128203,-0.298628,-0.416863,-0.089809,-1.719917,-1.459958,-0.151513,-0.014015,-0.381082,-0.517516,-0.500534,-0.13781,-1.374343,-1.051734,-0.58450,-0.578889,-0.43054,-0.009756,-0.035183,-0.539768,-0.193787,-0.758815,0.0
2,-0.077967,0.194582,0.182255,1.201905,0.052051,0.278332,-0.518745,0.118740,-1.428883,-0.226843,0.386893,0.120454,0.157091,1.689551,2.265430,2.321539,2.706000,2.732729,2.176746,2.209795,-0.667717,0.841472,2.631420,1.048463,2.361440,1.506456,0.026606,2.618557,-0.735931,-0.198135,-0.090953,-0.005188,-0.25889,-0.091427,-0.128203,-0.298628,-0.416863,-0.089809,0.503125,0.383082,-0.151513,-0.014015,-0.381082,-0.517516,0.176361,-0.13781,-0.031831,-1.229180,-0.46981,-0.578889,-0.43054,-0.009756,-0.035183,-0.390203,-0.005174,-0.758815,0.0
3,1.088516,0.194582,1.025636,-0.819754,-0.672908,-1.660719,-0.288537,-0.614595,-1.428883,0.238003,-1.470521,-7.285345,0.157091,-0.591873,-0.441417,-0.430749,-0.369549,-0.365935,-0.459401,-0.452531,0.774370,2.174127,2.631420,-0.598529,-0.437478,-0.378510,-0.481069,-0.674088,-0.735931,-0.198135,-0.090953,-0.005188,-0.25889,-0.091427,-0.128203,-0.298628,-0.256351,2.845824,0.301030,0.567386,-0.151513,-0.014015,-0.381082,-0.517516,-0.500534,-0.13781,-0.837338,-0.874288,-0.35512,-0.578889,-0.43054,-0.009756,-0.035087,-0.395135,-0.233636,-1.178727,0.0
4,-0.077967,0.194582,-0.661127,-0.683462,-0.645839,0.387777,-0.662010,-0.366208,0.699847,1.911451,0.851247,0.120454,0.157091,-0.591873,-0.441417,-0.430749,-0.369549,-0.365935,-0.459401,-0.452531,0.053326,-0.491182,-0.492897,-1.191372,-0.433989,-0.424673,-0.144179,-0.674088,1.497921,-0.198135,-0.090953,-0.005188,-0.25889,-0.091427,-0.128203,-0.298628,-0.416863,-0.089809,0.604172,-0.169830,-0.151513,-0.014015,-0.381082,-0.517516,-0.500534,-0.13781,0.057670,0.545278,-0.81388,0.231939,-0.24562,-0.009756,-0.035183,-0.536525,0.474591,0.500922,0.0


In [0]:
from sklearn.preprocessing import StandardScaler
#不用的特征
#进行数据归一化 mu=0 sigma=1
is_5g=train.is_5g
nouse=['area_id','user_id','prov_id','is_5g']
train=train.drop(nouse,axis=1)
train01= StandardScaler().fit_transform(train)
train= pd.DataFrame(train01,columns=train.columns)
train=pd.concat([train,is_5g],axis=1)
train.head()

,chnl_type,service_type,product_type,innet_months,total_times,total_flux,total_fee,pay_fee,sex,age,manu_name,term_type,max_rat_flag,is_5g_base_cover,is_work_5g_cover,is_home_5g_cover,is_work_5g_cover_l01,is_home_5g_cover_l01,is_work_5g_cover_l02,is_home_5g_cover_l02,activity_type,is_act_expire,comp_type,call_days,re_call10,short_call10,long_call10,bank_cnt,active_days01,active_days02,active_days03,active_days04,active_days05,active_days06,active_days07,active_days08,active_days09,active_days10,active_days11,active_days12,active_days13,active_days14,active_days15,active_days16,active_days17,active_days18,active_days19,active_days20,active_days21,active_days22,active_days23,game_app_flux,live_app_flux,video_app_flux,city_5g_ratio,city_level,is_5g
0,-0.077222,0.194507,1.025662,-0.501770,0.087459,0.871692,0.012404,-0.068877,0.700331,-0.878048,-0.542904,0.120061,0.157419,-0.591683,-0.441614,-0.430488,-0.369365,-0.365688,-0.459322,-0.452615,0.052984,-0.491712,-0.493179,0.606609,-0.426382,0.207429,-0.412696,-0.268993,-0.467388,-0.198019,-0.090597,-0.0045,-0.258437,-0.091642,-0.128275,-0.298889,-0.416503,-0.089798,0.503743,-0.261102,-0.151318,-0.014408,-0.38073,-0.517534,-0.049051,-0.138344,0.953467,-0.873243,-0.583635,-0.578601,-0.430343,-0.009748,-0.033011,0.129664,0.026629,0.919884,1.0
1,-0.077222,0.194507,-0.660076,-1.047385,-0.258518,-0.877114,-0.041401,-0.034188,0.700331,-0.785079,0.853670,0.120061,0.157419,-0.591683,-0.441614,-0.430488,-0.369365,-0.365688,-0.459322,-0.452615,0.052984,-0.491712,-0.493179,0.141559,-0.309562,-0.035447,0.022492,-0.673485,-1.093054,-0.198019,-0.090597,-0.0045,-0.258437,-0.091642,-0.128275,-0.298889,-0.416503,-0.089798,-1.717039,-1.458745,-0.151318,-0.014408,-0.38073,-0.517534,-0.500228,-0.138344,-1.372628,-1.050684,-0.583635,-0.578601,-0.430343,-0.009748,-0.033011,-0.540111,-0.193061,-0.760030,0.0
2,-0.077222,0.194507,0.182793,1.203279,0.052101,0.279143,-0.516068,0.116133,-1.427896,-0.227260,0.388145,0.120061,0.157419,1.690094,2.264421,2.322944,2.707346,2.734571,2.177124,2.209385,-0.668089,0.839405,2.629387,1.047572,2.359436,1.525713,0.024807,2.614459,-0.735531,-0.198019,-0.090597,-0.0045,-0.258437,-0.091642,-0.128275,-0.298889,-0.416503,-0.089798,0.503743,0.383782,-0.151318,-0.014408,-0.38073,-0.517534,0.176537,-0.138344,-0.030650,-1.228125,-0.468801,-0.578601,-0.430343,-0.009748,-0.033011,-0.390426,-0.004421,-0.760030,0.0
3,1.088542,0.194507,1.025662,-0.820045,-0.683350,-1.657195,-0.287324,-0.600783,-1.427896,0.237588,-1.473953,-7.301798,0.157419,-0.591683,-0.441614,-0.430488,-0.369365,-0.365688,-0.459322,-0.452615,0.774058,2.170521,2.629387,-0.599556,-0.438587,-0.383526,-0.477511,-0.673485,-0.735531,-0.198019,-0.090597,-0.0045,-0.258437,-0.091642,-0.128275,-0.298889,-0.255763,2.824029,0.301854,0.568035,-0.151318,-0.014408,-0.38073,-0.517534,-0.500228,-0.138344,-0.835837,-0.873243,-0.353966,-0.578601,-0.430343,-0.009748,-0.032923,-0.395363,-0.232916,-1.180009,0.0
4,-0.077222,0.194507,-0.660076,-0.683642,-0.655889,0.388436,-0.658422,-0.357957,0.700331,1.911042,0.853670,0.120061,0.157419,-0.591683,-0.441614,-0.430488,-0.369365,-0.365688,-0.459322,-0.452615,0.052984,-0.491712,-0.493179,-1.192448,-0.435100,-0.430283,-0.144176,-0.673485,1.498991,-0.198019,-0.090597,-0.0045,-0.258437,-0.091642,-0.128275,-0.298889,-0.416503,-0.089798,0.604687,-0.168976,-0.151318,-0.014408,-0.38073,-0.517534,-0.500228,-0.138344,0.058815,0.546286,-0.813304,0.231825,-0.245195,-0.009748,-0.033011,-0.536865,0.475414,0.499906,0.0


In [0]:
from sklearn.preprocessing import StandardScaler
#不用的特征
#进行数据归一化 mu=0 sigma=1
y_test=test.is_5g
nouse=['area_id','user_id','prov_id','is_5g']
x_test=test.drop(nouse,axis=1)
x_test01= StandardScaler().fit_transform(x_test)
x_test= pd.DataFrame(x_test01,columns=x_test.columns)
x_test.tail()

,chnl_type,service_type,product_type,innet_months,total_times,total_flux,total_fee,pay_fee,sex,age,manu_name,term_type,max_rat_flag,is_5g_base_cover,is_work_5g_cover,is_home_5g_cover,is_work_5g_cover_l01,is_home_5g_cover_l01,is_work_5g_cover_l02,is_home_5g_cover_l02,activity_type,is_act_expire,comp_type,call_days,re_call10,short_call10,long_call10,bank_cnt,active_days01,active_days02,active_days03,active_days04,active_days05,active_days06,active_days07,active_days08,active_days09,active_days10,active_days11,active_days12,active_days13,active_days14,active_days15,active_days16,active_days17,active_days18,active_days19,active_days20,active_days21,active_days22,active_days23,game_app_flux,live_app_flux,video_app_flux,city_5g_ratio,city_level
199995,-0.079835,0.194767,1.025574,-0.909705,1.137910,0.204357,-0.688540,-0.326702,-1.431356,-0.411737,0.845267,0.12143,0.156271,-0.592348,-0.440925,-0.4314,-0.370008,-0.366550,-0.459601,-0.452321,0.054181,-0.489859,-0.492190,1.102558,5.130991,1.400426,2.000899,-0.675608,-1.094083,-0.198426,-0.09184,-0.006669,-0.260031,-0.090885,-0.128026,-0.297979,-0.417766,-0.089861,0.602885,0.842412,-0.152006,-0.013206,-0.381962,-0.517470,-0.501300,-0.136502,-0.482740,-1.231822,-0.015002,-0.579612,-0.431034,-0.010511,-0.044053,-0.332034,-0.613932,0.923211
199996,-0.079835,0.194767,1.870242,-0.252313,0.591794,0.172855,-0.429794,-0.263808,0.698638,1.819505,0.383802,0.12143,0.156271,-0.592348,-0.440925,-0.4314,-0.370008,-0.366550,-0.459601,-0.452321,0.054181,-0.489859,-0.492190,1.048842,-0.061155,1.926476,0.432910,-0.675608,-0.647646,-0.198426,-0.09184,-0.006669,-0.260031,-0.090885,-0.128026,-0.297979,-0.417766,-0.089861,0.602885,0.289113,-0.152006,-0.013206,-0.381962,-0.517470,-0.501300,-0.136502,-1.289055,-1.231822,-0.815328,-0.579612,-0.431034,-0.010511,-0.044053,-0.534256,-0.466287,0.083715
199997,1.088454,0.194767,-1.508428,-0.456331,-0.694042,-0.454505,-0.393017,-0.766963,0.698638,0.517947,1.306731,0.12143,0.156271,-0.592348,-0.440925,-0.4314,-0.370008,-0.366550,-0.459601,-0.452321,-2.108725,2.183203,2.636523,-0.699690,-0.434703,-0.305819,-0.490569,-0.675608,0.959527,-0.198426,-0.09184,-0.006669,-0.260031,-0.090885,-0.128026,-0.297979,-0.417766,-0.089861,0.602885,-0.633051,-0.152006,-0.013206,-0.381962,-0.517470,-0.501300,-0.136502,-1.199464,-0.433260,-0.815328,0.638143,-0.246682,-0.010511,-0.044053,-0.538916,-0.664400,0.923211
199998,1.088454,0.194767,-1.508428,-1.068386,-0.049257,0.502119,-0.672201,0.994079,0.698638,2.191379,-1.000591,0.12143,0.156271,-0.592348,-0.440925,-0.4314,-0.370008,-0.366550,-0.459601,-0.452321,-0.666788,2.183203,-0.492190,-0.375545,-0.434703,-0.314524,0.182801,-0.658186,1.584539,0.974532,-0.09184,-0.006669,-0.260031,-0.090885,-0.128026,-0.297979,-0.417766,-0.089861,0.602885,0.842412,-0.152006,-0.013206,-0.381962,-0.517470,-0.275561,-0.136502,1.129888,1.430053,-0.700996,-0.579612,-0.246682,-0.010511,-0.044052,0.012512,-0.469962,0.923211
199999,-2.416414,0.194767,1.025574,0.495754,0.286938,0.375910,0.118445,0.126137,0.698638,-0.690643,0.383802,0.12143,0.156271,1.688195,-0.440925,-0.4314,-0.370008,2.728138,-0.459601,-0.452321,0.054181,-0.489859,0.133552,0.826571,-0.179853,-0.038441,-0.201982,-0.330475,1.316677,-0.198426,-0.09184,-0.006669,-0.260031,-0.090885,-0.128026,4.820894,0.062082,-0.089861,0.602885,0.842412,-0.152006,-0.013206,-0.381962,-0.376035,-0.275561,-0.136502,-0.303559,1.163866,1.356987,-0.579612,-0.431034,-0.010511,-0.037840,-0.538903,0.904107,-1.595279


In [0]:
## 下采样
# 分离特征和标签
x_train = train.loc[:, train.columns != 'is_5g']
y_train = train['is_5g']
# 计算少数类个数
number_records_is_5g = len(train[train.is_5g == 1])
# 取得少数类样本的索引
is_5g_indices = np.array(train[train.is_5g == 1].index)
# 取得多数类样本的索引
normal_indices = train[train.is_5g == 0].index

# 从多数类中随机选择与少数类个数相同的样本数作为样本
random_normal_indices = np.random.choice(normal_indices, number_records_is_5g, replace = False)
random_normal_indices = np.array(random_normal_indices)

# 合并随机取得的0类和全部的1类的索引
under_sample_indices = np.concatenate([is_5g_indices,random_normal_indices])

# 根据索引得到下采样后的数据集
under_sample_train = train.iloc[under_sample_indices,:]

# 分离特征和标签
x_undersample = under_sample_train.iloc[:, under_sample_train.columns != 'is_5g']
y_undersample = under_sample_train.iloc[:, under_sample_train.columns == 'is_5g']

print("正样本比例(0类): ", len(under_sample_train[under_sample_train.is_5g == 0])/len(under_sample_train))
print("负样本比例(1类): ", len(under_sample_train[under_sample_train.is_5g == 1])/len(under_sample_train))
print("下采样后总样本个数为: ", len(under_sample_train))

正样本比例(0类):  0.5
负样本比例(1类):  0.5
下采样后总样本个数为:  13362


In [0]:
train_data,test_data, train_labels, test_labels=\
x_undersample.values, x_test.values, y_undersample.values, y_test.values

In [0]:
#随机森林方法
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=2, random_state=0)
model.fit(train_data,train_labels)
print("测试集:")
predict_target2=model.predict(test_data)
print("预测正确数量,测试集样本量:")
print(sum(predict_target2 == test_labels),len(test_labels))
print("精确度等指标：")
print(metrics.classification_report(test_labels,predict_target2))
print("混淆矩阵：")
print(metrics.confusion_matrix(test_labels,predict_target2))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


测试集:
预测正确数量,测试集样本量:
145542 200000
精确度等指标：
              precision    recall  f1-score   support

         0.0       1.00      0.73      0.84    197408
         1.0       0.04      0.84      0.07      2592

    accuracy                           0.73    200000
   macro avg       0.52      0.78      0.46    200000
weighted avg       0.98      0.73      0.83    200000

混淆矩阵：
[[143367  54041]
 [   417   2175]]


In [0]:
#logistic回归
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=0,l1_ratio=0.01)
model.fit(train_data,train_labels)
print("测试集:")
predict_target2=model.predict(test_data)
print("预测正确数量,测试集样本量:")
print(sum(predict_target2 == test_labels),len(test_labels))
print("精确度等指标：")
print(metrics.classification_report(test_labels,predict_target2))
print("混淆矩阵：")
print(metrics.confusion_matrix(test_labels,predict_target2))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1501: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


测试集:
预测正确数量,测试集样本量:
156455 200000
精确度等指标：
              precision    recall  f1-score   support

         0.0       1.00      0.78      0.88    197408
         1.0       0.04      0.74      0.08      2592

    accuracy                           0.78    200000
   macro avg       0.52      0.76      0.48    200000
weighted avg       0.98      0.78      0.87    200000

混淆矩阵：
[[154538  42870]
 [   675   1917]]


In [0]:
from sklearn.ensemble import GradientBoostingClassifier
model =GradientBoostingClassifier(random_state=123)
model.fit(train_data,train_labels)
print("测试集:")
predict_target2=model.predict(test_data)
print("预测正确数量,测试集样本量:")
print(sum(predict_target2 == test_labels),len(test_labels))
print("精确度等指标：")
print(metrics.classification_report(test_labels,predict_target2))
print("混淆矩阵：")
print(metrics.confusion_matrix(test_labels,predict_target2))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


测试集:
预测正确数量,测试集样本量:
157574 200000
精确度等指标：
              precision    recall  f1-score   support

         0.0       1.00      0.79      0.88    197408
         1.0       0.05      0.84      0.09      2592

    accuracy                           0.79    200000
   macro avg       0.52      0.81      0.49    200000
weighted avg       0.99      0.79      0.87    200000

混淆矩阵：
[[155397  42011]
 [   415   2177]]


In [0]:
#使用adaboost分类算法
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state=0)
model.fit(train_data,train_labels)
print("测试集:")
predict_target2=model.predict(test_data)
print("预测正确数量,测试集样本量:")
print(sum(predict_target2 == test_labels),len(test_labels))
print("精确度等指标：")
print(metrics.classification_report(test_labels,predict_target2))
print("混淆矩阵：")
print(metrics.confusion_matrix(test_labels,predict_target2))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


测试集:
预测正确数量,测试集样本量:
156467 200000
精确度等指标：
              precision    recall  f1-score   support

         0.0       1.00      0.78      0.88    197408
         1.0       0.05      0.83      0.09      2592

    accuracy                           0.78    200000
   macro avg       0.52      0.81      0.48    200000
weighted avg       0.98      0.78      0.87    200000

混淆矩阵：
[[154317  43091]
 [   442   2150]]
